In [121]:
#!/usr/bin/env python3
"""Show a text-mode spectrogram using live microphone data."""
import argparse
import logging
import numpy as np
import time
import soundfile
import itertools

DURATION=20
gain=10
range=[100,2000]
high=2000
low=100

ar=[]
au=[]
try:
    import sounddevice as sd

    #samplerate = sd.query_devices(args.device, 'input')['default_samplerate']
    samplerate = 16000.0
    
    delta_f = (high - low) / (80 - 1)
    fftsize = np.ceil(samplerate / delta_f).astype(int)
    low_bin = int(np.floor(low / delta_f))

    cumulated_status = sd.CallbackFlags()

    def callback(indata, frames, time, status):
        global cumulated_status
        cumulated_status |= status
        if any(indata):
            magnitude = np.abs(np.fft.rfft(indata[:, 0], n=fftsize))
            magnitude *= gain / fftsize
            au.extend(itertools.chain(indata.tolist()))
            ar.append(magnitude)
            #line = (gradient[int(np.clip(x, 0, 1) * (len(gradient) - 1))]
            #        for x in magnitude[low_bin:low_bin + 80])
            #print('.')
            #print(*line, sep='', end='\x1b[0m\n', flush=True)
        else:
            #print('no input', flush=True)
            pass
    
    with sd.InputStream(device=None, channels=1, callback=callback,
                        blocksize=int(samplerate * DURATION / 1000),
                        samplerate=samplerate):
        while True:
            #response = input()
            #if response in ('', 'q', 'Q'):
            time.sleep(1)
            break
            
    if cumulated_status:
        logging.warning(str(cumulated_status))
except Exception as e:
    print(e)

In [122]:
import matplotlib.pyplot as plt
type(ar[0])

numpy.ndarray

In [123]:
len(au)

15360

In [124]:
import soundfile

In [125]:
soundfile.write('test2.wav', au , 16000)

In [126]:
import IPython.display as ipd
ipd.Audio('test.wav')

In [127]:
ipd.Audio('test2.wav')

In [128]:
aar, sr = soundfile.read('test.wav')
aar2, sr = soundfile.read('test2.wav')

In [135]:
output = 0.5 * aar + 0.5 * aar2

In [136]:
soundfile.write('testres.wav', output, 16000)

In [137]:
ipd.Audio('testres.wav')